In [2]:
import pandas as pa
import numpy as np
import random
import csv
import math 
with open("D:/Download/iris.csv", newline= "", encoding="ISO-8859-1") as filecsv:  #Importo i dati dal file csv
    DATI = np.asmatrix(pa.read_csv("D:/Download/iris.csv", header=None))
    XComplete = np.array(DATI[:,0:4])
    yComplete = np.array(DATI[:,4]) 
#print(XComplete)
#print(yComplete)
num_labels = 3
[m,n] = XComplete.shape                                      
l = list(range(m))
random.shuffle(l)
XMisto = XComplete[l,:]                                 #Mischio i dati
yMisto = yComplete[l]
#print(XMisto)
#print(yMisto)
size_train = math.ceil(m * 0.6)                        
size_cv = math.ceil(m * 0.2)
X_ts = XMisto[0:size_train,:]
y_ts = yMisto[0:size_train]
#print(X_ts.shape)
#print(y_ts.shape)                                      #Costrusco il training set(60%), il cv set(20%) e il test set(20%)
X_cv = XMisto[size_train : size_train+size_cv,:]
y_cv = yMisto[size_train : size_train+size_cv]
#print(X_cv.shape)
#print(y_cv.shape)
X_test = XMisto[size_train+size_cv:m,:]
y_test = yMisto[size_train+size_cv:m]
#print(x_test.shape)
#print(y_test.shape)


def featureNormalize(X):
    X_norm = X
    mu = X.mean(0)
    sigma = X.std(0)                                              #Implemento la feature normalize
    r,c = X.shape
    for i in range(c):
        X_norm[:,i] = ((X[:,i] - mu[i]) / sigma[i])
    return X_norm

X_ts_norm = featureNormalize(X_ts)
#print(X_ts_norm)
X_cv_norm = featureNormalize(X_cv)
X_test_norm = featureNormalize(X_test)

dimTotClassi = 30;
valoreInferiore = -3.2;
valoreSuperiore = 3.2;
step = (abs(valoreInferiore)+abs(valoreSuperiore))/dimTotClassi
intervallo = np.array(list(np.arange(valoreInferiore, valoreSuperiore+0.1, step)))

def learningBayes(X, y, pattern, classe):
    num_eventi = sum(pattern)
    [m, n] = X.shape
    contatore = 0                               #Funzione per calcolare le verosomiglianze per ogni evento
    p = np.zeros(num_eventi)
    for i in range(0, n):
        for j in range(0,pattern[i]):
            p[contatore] = (np.sum(X[np.nonzero(np.array(y[y==classe]))]==j-1)+1) / (np.shape(np.array(y[y== classe]))[0]+pattern[i])
            contatore = contatore + 1
    return p

def transform(X, intervallo):
    [m,n] = X.shape
    for i in range(0,m):                                          #Discretizzo i dati
        for j in range(0,n):
            for k in range(0, (np.shape(intervallo)[0]-1)):
                if X[i,j] > intervallo[k] and X[i,j] < intervallo[k + 1]:
                    X[i,j] = k
                    
    return X

X_ts_norm = transform(X_ts_norm,intervallo).astype(int)
#print(X_ts_norm)
X_cv_norm = transform(X_cv_norm, intervallo).astype(int)
X_test_norm = transform(X_test_norm, intervallo).astype(int)

pattern = np.array([30,30,30,30])
p1 = learningBayes(X_ts_norm, y_ts, pattern, 1)
#print(p1)
p2 = learningBayes(X_ts_norm, y_ts, pattern, 2)
p3 = learningBayes(X_ts_norm, y_ts, pattern, 3)
#print(p2, p3)

p1Smoth = (np.size(yMisto[yMisto== 1])) / m+3  
p2Smoth = (np.size(yMisto[yMisto== 2])) / m+3                #Eseguo lo smoothing
p3Smoth = (np.size(yMisto[yMisto== 3])) / m+3

#print(p1Smoth)
#print(p2Smoth)
#print(p3Smoth)

predizione1 = np.zeros(30);
predizione2 = np.zeros(30);
predizione3 = np.zeros(30);

def previsioni(X_test, prob, pattern, p_class):
    contatore = 0
    p = np.zeros(np.shape(X_test)[0])
    for i in range(0, np.shape(pattern)[0]):              #funzione per le previsioni
        p[i] = prob[contatore + X_test[i]]
        contatore = contatore + pattern[i]
    p_sum = math.exp(np.sum(np.log(np.transpose(p))))
    pred = p_sum * p_class;
    return pred

for j in range(0,30):
    predizione1[j] = previsioni(X_test_norm[j, :], p1, pattern, p1Smoth)
    predizione2[j] = previsioni(X_test_norm[j, :], p2, pattern, p2Smoth)
    predizione3[j] = previsioni(X_test_norm[j, :], p3, pattern, p3Smoth)

predizioni = np.array([np.transpose(predizione1),np.transpose(predizione2),np.transpose(predizione3)])
#print(predizioni)

predClasse1 = (np.amax(predizioni, axis=0) == np.transpose(predizione1)).astype(int) * 1
predClasse2 = (np.amax(predizioni, axis=0) == np.transpose(predizione2)).astype(int) * 2 #per ogni classe la sua previsione
predClasse3 = (np.amax(predizioni, axis=0) == np.transpose(predizione3)).astype(int) * 3

#print(probClasse1)

risultati = np.transpose(np.array([np.transpose(predClasse1) + np.transpose(predClasse2) + np.transpose(predClasse3)]))
print(risultati)

accuratezza = np.size(y_test[y_test == risultati])/np.size(y_test)     #Calcolo accuratezza
print(accuratezza)



[[3]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [3]
 [2]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [2]
 [1]
 [1]
 [3]]
0.5666666666666667
